# Hyperparameteroptimierung

Dieses Skript dient zum Finden der passenden Hyperparameter zum Trainieren des GRU-Modells

Imports

In [ ]:
import pandas as pd
import pickle 
import re

from sklearn.preprocessing import MinMaxScaler
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from kerastuner import HyperModel, RandomSearch, HyperParameters

from utils import create_daily_sequences, test_sequences, get_optimization_results

### Loading Data

In [ ]:
with open('Data/heatpump/data_heatpump_cleaned_v1.pkl', 'rb') as f:
    load_dict = pickle.load(f)

with open('Data/weather/data_weather_v1.pkl', 'rb') as f:
    weather_data = pickle.load(f)

building_info = pd.read_excel("Data/Gebaeudeinformationen_cleaned.xlsx", index_col=0)
building_info.set_index("Building number", inplace=True)

# add building information
for house in load_dict:
    id = int(re.findall(r'\d+', house)[0])

    load_dict[house]["area"] = building_info.loc[id]["Building area"]
    load_dict[house]["inhabitants"] = building_info.loc[id]["Number of inhabitants"]
    load_dict[house]["building"] = id
    
    weather_data = weather_data[weather_data.index>=1528965900]
    load_dict[house] = pd.concat([load_dict[house], weather_data], axis=1)

### Prepare Data for Modelling

In [ ]:
# Verwendung eines Datensatu, um Berechnungszeit einzusparen
df = load_dict["SFH23"] 

# Skalierung der Daten
len_input_seq = 3       # in Tagen
len_output_seq = 1      # in Tagen
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
X, y = create_daily_sequences(df_scaled, len_input_seq, len_output_seq)
test_sequences(X, y, len_input_seq*96, len_output_seq*96)

In [ ]:
# Aufteilung in Trainings und Validierungsdatensatz

train_size = int(len(X) * 0.8)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size], y[train_size:]

# Anpassen des Zielvariablen-Formats an Modellarchitektur (192 Ausgabeneuronen, keine zweidimensionaler Aufbau)
y_train_reshaped = y_train.reshape(-1, 96 * 2)  # Umformen in [Anzahl der Beispiele, 192]
y_val_reshaped = y_val.reshape(-1, 96 * 2)      # Umformen in [Anzahl der Beispiele, 192]

### Optimierungsansatz 1

In [ ]:
class GRUHyperModel(HyperModel):
    def __init__(self, input_shape, output_units):
        self.input_shape = input_shape
        self.output_units = output_units

    def build(self, hp):
        model = Sequential()
        
        for i in range(hp.Int('num_gru_layers', 1, 5)):
            if i == 0:
                # Nur für die erste Schicht wird input_shape gesetzt
                model.add(GRU(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                              return_sequences=True if hp.get('num_gru_layers') > 1 else False,
                              input_shape=self.input_shape))
            else:
                # Für nachfolgende Schichten wird input_shape nicht gesetzt
                model.add(GRU(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                              return_sequences=True if i < hp.get('num_gru_layers') - 1 else False))

            model.add(Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))

        model.add(Dense(self.output_units, activation='linear'))
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    def fit(self, hp, model, *args, **kwargs):
        # Anpassung der Batch-Größe als Hyperparameter
        kwargs['batch_size'] = hp.Choice('batch_size', values=[32, 64, 128])
        return model.fit(*args, **kwargs)

In [ ]:
# Eingabe- und Ausgabe-Shape für das Modell
input_shape = (None, 13)    # Zum Beispiel (Sequenzlänge, Anzahl der Merkmale)
output_units = 96 * 2       # Zum Beispiel (Sequenzlänge * Anzahl der Zielvariablen)
tuner = RandomSearch(
    GRUHyperModel(input_shape, output_units),
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='my_dir',
    project_name='gru_hyperparam_tuning_3'
)

tuner.search(x=X_train, 
             y=y_train_reshaped, 
             epochs=10, 
             validation_data=(X_val, y_val_reshaped))

In [ ]:
results = get_optimization_results(tuner, 20)
results[results['val_loss'] == results['val_loss'].min()]

### Optimierungsansatz 2

In [ ]:
class GRUHyperModel(HyperModel):
    def __init__(self, input_shape, output_units):
        self.input_shape = input_shape
        self.output_units = output_units

    def build(self, hp):
        model = Sequential()
        
        # Eingabeschicht
        model.add(GRU(
            units=hp.get('units_0'),
            return_sequences=hp.get('num_gru_layers') > 1,
            input_shape=self.input_shape
        ))
        model.add(Dropout(hp.get('dropout_rate_0')))
        
        # Weitere GRU Schichten
        for i in range(1, hp.get('num_gru_layers')):
            model.add(GRU(
                units=hp.get(f'units_{i}'),
                return_sequences=i < hp.get('num_gru_layers') - 1
            ))
            model.add(Dropout(hp.get(f'dropout_rate_{i}')))

        # Ausgabeschicht
        model.add(Dense(self.output_units, activation='linear'))
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

In [ ]:
# Festlegen der Hyperparameter außerhalb des Hypermodells
hp = HyperParameters()
hp.Int('num_gru_layers', 1, 5, default=2)
hp.Int('units_0', 32, 256, step=32)
hp.Float('dropout_rate_0', 0, 0.5, step=0.1)
for i in range(1, 5):  # Maximal 5 Schichten
    hp.Int(f'units_{i}', 32, 256, step=32, default=32)
    hp.Float(f'dropout_rate_{i}', 0, 0.5, step=0.1, default=0.1)
hp.Choice('batch_size', values=[32, 64, 128])

# Eingabe- und Ausgabe-Shape für das Modell
input_shape = (None, 13)  # Zum Beispiel (Sequenzlänge, Anzahl der Merkmale)
output_units = 96 * 2  # Zum Beispiel (Sequenzlänge * Anzahl der Zielvariablen)

# Initialisieren des Tuners
tuner = RandomSearch(
    GRUHyperModel(input_shape, output_units),
    hyperparameters=hp,
    objective='val_loss',
    max_trials=100,
    executions_per_trial=1,
    directory='my_dir',
    project_name='gru_hyperparam_tuning_6'
)

# Starten des Suchvorgangs
tuner.search(
    x=X_train,
    y=y_train_reshaped,
    epochs=10,
    validation_data=(X_val, y_val_reshaped),
    batch_size=hp.get('batch_size')  # Verwendung der Batch-Größe aus den Hyperparametern
)

In [ ]:
results = get_optimization_results(tuner, 20)
results[results['val_loss'] == results['val_loss'].min()]

In [ ]:
import plotly.graph_objects as go

# Erstellen eines Lineplots mit Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results.index,
    y=results['val_loss'],
    mode='lines+markers',
    name='Validierungsverlust'
))

# Update layout für Titel und Gitternetz
fig.update_layout(
    title={
        'text': 'Entwicklung des Validierungsverlustes über die Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Trial Index',
    yaxis_title='Validierungsverlust',
    template='simple_white',
    xaxis=dict(showgrid=True),  # Gitternetz für die X-Achse
    yaxis=dict(showgrid=True)   # Gitternetz für die Y-Achse
)

fig.show()